In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os

extracted_data_path = '/content/drive/MyDrive/refer-kitti.7z'
if os.path.exists(extracted_data_path):
    os.system(f'7z x {extracted_data_path} -o/content/drive/MyDrive/ -y')
    